<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Activity" data-toc-modified-id="Activity-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Activity</a></span></li><li><span><a href="#Libraries" data-toc-modified-id="Libraries-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Libraries</a></span></li><li><span><a href="#Setting-up-the-data" data-toc-modified-id="Setting-up-the-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Setting up the data</a></span></li><li><span><a href="#Modeling-changes-to-a-tissue-during-infection" data-toc-modified-id="Modeling-changes-to-a-tissue-during-infection-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Modeling changes to a tissue during infection</a></span></li><li><span><a href="#Run-simulation-and-visualize" data-toc-modified-id="Run-simulation-and-visualize-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Run simulation and visualize</a></span></li><li><span><a href="#How-immune-cells-affect-infection" data-toc-modified-id="How-immune-cells-affect-infection-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>How immune cells affect infection</a></span></li><li><span><a href="#Plot-infection-with-immune-system-over-time" data-toc-modified-id="Plot-infection-with-immune-system-over-time-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Plot infection with immune system over time</a></span></li></ul></div>

# Activity

    
    Imagine a tissue with n x n cells with different states:
    
    H: Healthy
    VA: Viral assembly
    VR: Viral release
    D: Dead
    
    INFECT_PROB = probability of infection (i.e. the likelihood of a cell becoming infected)
    
    

# Libraries

In [1]:
#Importing libraries: computer code 'cookbooks'

import matplotlib.pyplot
import pandas as pd
import numpy as np
import random
import plotly as py
import plotly.graph_objs as go

# Setting up the data

In [13]:
INFECT_PROB = 0 

n = 5 # size of tissue
tissue = pd.DataFrame("H", index = range(n), columns = range(n)) # write empty data table



max_starting_infected = 5 # you choose maximum number of starting infected cells
n_starting_infected = random.randint(0, max_starting_infected) # random number generated for of starting number infected cells



# choose (x,y) where to place the infected cells in tissue
points = set()
while len(points) < n_starting_infected:
    points.add((random.randint(0, n-1), random.randint(0, n-1))) 

    
    
# set these points to "VR" for "infected"
for point in points:
    tissue.iloc[point[0], point[1]] = "VR"
    
print("# starting infected cells: {}".format(n_starting_infected))

points

# starting infected cells: 5


{(1, 0), (1, 3), (1, 4), (2, 1), (2, 2)}

In [14]:
tissue

,0,1,2,3,4
0,H,H,H,H,H
1,VR,H,H,VR,VR
2,H,VR,VR,H,H
3,H,H,H,H,H
4,H,H,H,H,H


# Modeling changes to a tissue during infection

    Next stages for:
    
        Healthy cell: 
            If adjacent to cells releasing viruses (i.e. "VR"), give a random probability between 0 and 1.
            If probability <= INFECT_PROB, healthy cell is infected and becomes H -> VA
            
        Viral assembly:
            Release of viruses (i.e. VA -> VR)
            
        Viral release:
            Dies (i.e. VR -> D)
            
        Dead cell:
            No change
            
            

In [4]:
def one_time_step(tissue, infect_prob):                      #Q2: how many times does this run
    
    updated_tissue = tissue.copy()
    for row in range(n):
        for col in range(n):

            # make changes to healthy cell
            if tissue.iloc[row, col] == "H":
                if row != n-1:
                    if tissue.iloc[row+1,col] == "VR":
                        if random.random() < infect_prob:
                            updated_tissue.iloc[row,col] = "VA"

                if col != n-1:       
                    if tissue.iloc[row,col+1] == "VR":
                        if random.random() < infect_prob:
                            updated_tissue.iloc[row,col] = "VA"

                if row != 0:
                    if tissue.iloc[row-1,col] == "VR":
                        if random.random() < infect_prob:
                            updated_tissue.iloc[row,col] = "VA"

                if col != 0:
                    if tissue.iloc[row,col-1] == "VR":
                        if random.random() < infect_prob:
                            updated_tissue.iloc[row,col] = "VA"

            # make changes to newly infected cell
            if tissue.iloc[row, col] == "VA":
                updated_tissue.iloc[row, col] = "VR"
            
            # make changes to infectious cell 
            if tissue.iloc[row, col] == "VR":
                updated_tissue.iloc[row, col] = "D"
    
            # make changes to dead cell
            if tissue.iloc[row, col] == "D":
                updated_tissue.iloc[row, col] = "D" 
                
    return(updated_tissue)
            
def t_time_step(t, tissue, infect_prob):                    #Q2 When you call this function how many times does above run
    
    out = []
    out.append(tissue.values)
    
    updated_tissue = tissue.copy()
    for step in range(t):
        updated_tissue = one_time_step(updated_tissue, infect_prob)
        out.append(updated_tissue.values)
        
    return(out)
        


# Run simulation and visualize

In [79]:
#TODO: CHANGE PARAMETERS HERE
INFECT_PROB = 0.5                                                 # Probality VR cell next to H cell infects it: Q4a
n = 30                                                            # size of tissue
max_starting_infected = 5                                         # choose maximum number of starting infected cells
n_time_steps = 20                                                 # choose number of steps in movie


#Set up tissue based on paramters above
# Changed to make sure we always start with at least 1 infected cell

tissue = pd.DataFrame("H", index = range(n), columns = range(n)) # write empty data table
n_starting_infected = random.randint(1, max_starting_infected)   # choose random number of starting infected cells


points = set()
while len(points) < n_starting_infected:   
    points.add((random.randint(0, n-1), random.randint(0, n-1)))# (x,y) where to place the infected cells in tissue


for point in points:                                            # set these points to "VR" for relasing virus 
    tissue.iloc[point[0], point[1]] = "VR"
    
print("# starting infected cells: {}".format(n_starting_infected))

tissue.iloc[0, 0] = "D"

# Run the simulation for n_time_steps
tissue_frames = t_time_step(n_time_steps, tissue, infect_prob = INFECT_PROB) 

# starting infected cells: 11


In [81]:
#Plot simulation - do not worry about what is happening in code here, just what movie it makes
numerify = {'H': 0, 'VA': 0.25, 'VR': 0.5, 'D': 0.75}

legend_colors = [
    [0, "#56B4E9"],
    [0.25,  "#56B4E9"],
    [0.25, "#F0E442"],
    [0.5, "#F0E442"],
    [0.5, "#D55E00"],
    [0.75, "#D55E00"],
    [0.75, "#000000"],
    [1.0, "#000000"]
]

numbers = [0, 0.25, 0.5, 0.75]
names   = ['Healthy', 'Viral Assembly', 'Viral Release', 'Dead']


# Numerify the cells for plotting
tissue_frames_numeric = []
for t in range(len(tissue_frames)):
    tissue_frames_numeric.append(tissue_frames[t].copy())
    for i in range(tissue_frames[t].shape[0]):
        for j in range(tissue_frames[t].shape[1]):
            tissue_frames_numeric[t][i][j] = numerify[tissue_frames[t][i][j]]
            
fig = go.Figure(data=go.Heatmap(z=tissue_frames_numeric[0], colorscale=legend_colors, colorbar=dict(tickvals=numbers, ticktext=names, thickness=25)),
               frames=[go.Frame(data=go.Heatmap(z=tissue_frames_numeric[i])) for i in range(len(tissue_frames_numeric))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()

# How immune cells affect infection

In [69]:
n = 10                                 # size of tissue
max_starting_infected = 5              # maximum number of starting infected cells 
n_immune_starting = 4                  # number of immune cells starting in tissue *** new ***
n_immune_recruited  = 0                # set number of immune cells recruited at first/each contact *** new ***
INFECT_PROB = 0.8                      # Probablity healthy cell will be infected by a VR cell if next to it

tissue = pd.DataFrame('H', index = range(n), columns = range(n))
n_starting_infected = random.randint(0, max_starting_infected) # choose number of starting infected cells
print("# starting infected cells: {}".format(n_starting_infected))


# Add infected cells
infected_points = set()
while len(infected_points) < n_starting_infected:
    infected_points.add((random.randint(0, n-1), random.randint(0, n-1)))
for point in infected_points:
    tissue.iloc[point[0], point[1]] = 'VR'
    
infected_points


# Add single immune cell *** new ***
immune_cell = (random.randint(0, n-1), random.randint(0, n-1))
tissue.iloc[immune_cell[0], immune_cell[1]] = 'I'
print(immune_cell)

# starting infected cells: 0
(3, 6)


In [70]:
#*** new ***
def immune_time_step(tissue):
    updated_tissue = tissue.copy()
    foundInfection = False
    for row in range(n):
        for col in range(n):
            if tissue.iloc[row, col] == 'I':
                #eliminate virus infected cells
                if row != n-1:
                    if tissue.iloc[row+1,col] in ['VA','VR']:
                        updated_tissue.iloc[row+1,col] = 'D'
                        foundInfection = True

                if col != n-1:       
                    if tissue.iloc[row,col+1]in ['VA','VR']:
                        updated_tissue.iloc[row,col+1] = 'D'
                        foundInfection = True

                if row != 0:
                    if tissue.iloc[row-1,col] in ['VA','VR']:
                        updated_tissue.iloc[row-1,col] = 'D'
                        foundInfection = True

                if col != 0:
                    if tissue.iloc[row,col-1] in ['VA','VR']:
                        updated_tissue.iloc[row,col-1] = 'D'
                        foundInfection = True
                
                # move immune cell
                updated_tissue.iloc[row,col] = "H" # healthy cell fills in where immune cell was
    
                add2row = random.choice([-1, 0, 1]) # choose if immune down, same row, up
                add2col = random.choice([-1, 0, 1]) # choose if immune left, same column, right
    
                if add2row + row < 1:
                    add2row = 1
                if add2row + row >= n:
                    add2row = -1
                if add2col + col < 1:
                    add2col = 1
                if add2col + col >= n:
                    add2col = -1
                updated_tissue.iloc[add2row + row, add2col + col] = "I"
                
        if foundInfection == True:
            inmmune_cells_recruited = set()
            while len(inmmune_cells_recruited) < n_immune_recruited:
                inmmune_cells_recruited.add((random.randint(0, n-1), random.randint(0, n-1)))
            for point in inmmune_cells_recruited:
                updated_tissue.iloc[point[0], point[1]] = 'I'
            
    
    return(updated_tissue)

In [71]:
def t_time_step(t, tissue, infect_prob):
    
    out = []
    
    out.append(tissue.values)
    
    updated_tissue = tissue.copy()
    for step in range(t):
        updated_tissue = immune_time_step(updated_tissue)             #immune cells move and look for virus *** new ***
        updated_tissue = one_time_step(updated_tissue, infect_prob)   #virus infects and kills cells as before
        out.append(updated_tissue.values)
        
    return(out)
        

# if there is a any cell that is infectious, then there is some probability that it becomes infeccted (50%)

# Plot infection with immune system over time

In [84]:
#TODO: CHANGE PARAMETERS HERE

n = 30                     # size of tissue
max_starting_infected = 5  # maximum number of starting infected cells
n_immune_starting = 4      # number of immune cells starting in tissue
n_immune_recruited  = 1    # set number of immune cells recruited at first/each contact      Q7 - change here
INFECT_PROB = 0.5          # infection probablity
n_time_steps = 30


#Set up tissue based on paramters above

tissue = pd.DataFrame('H', index = range(n), columns = range(n))

# Add infected cells
n_starting_infected = random.randint(0, max_starting_infected) # choose number of starting infected cells

infected_points = set()
while len(infected_points) < n_starting_infected:
    infected_points.add((random.randint(0, n-1), random.randint(0, n-1)))
for point in infected_points:
    tissue.iloc[point[0], point[1]] = 'VR'
    
print("Starting infected cells: {}".format(n_starting_infected))
    

# Add n_immune_starting immune cells
immue_cell_points = set()

while len(immue_cell_points) < n_immune_starting:
    immue_cell_points.add((random.randint(0, n-1), random.randint(0, n-1)))
    
for point in immue_cell_points:
    tissue.iloc[point[0], point[1]] = 'I'
    
print("Immune cells", immue_cell_points)



#Run simulation
tissue_frames = t_time_step(n_time_steps, tissue, infect_prob = INFECT_PROB) 

Starting infected cells: 3
Immune cells {(22, 27), (29, 17), (16, 19), (24, 3)}


In [87]:
#Plot simulation - do not worry about what is happening in code here, just what movie it makes
numerify = {'H': 0, 'VA': 0.2, 'VR': 0.4, 'D': 0.6, 'I': 0.8}

tissue_frames_numeric = []
for t in range(len(tissue_frames)):
    tissue_frames_numeric.append(tissue_frames[t].copy())
    for i in range(tissue_frames[t].shape[0]):
        for j in range(tissue_frames[t].shape[1]):
            tmp = numerify[tissue_frames[t][i][j]]
            tissue_frames_numeric[t][i][j] = tmp

legend_colors = [
    [0, "#0072B2"],
    [0.2,  "#0072B2"],
    [0.2, "#F0E442"],
    [0.4, "#F0E442"],
    [0.4, "#D55E00"],
    [0.6, "#D55E00"],
    [0.6, "#000000"],
    [0.8, "#000000"],
    [0.8, "#00FF3A"],
    [1.0, "#00FF3A"]
]

numbers = [0, 0.2, 0.4, 0.6, 0.8]
names = ['Healthy', 'Viral Assembly', 'Viral Release', 'Dead', 'Immune']


fig = go.Figure(data=go.Heatmap(z=tissue_frames_numeric[0], colorscale=legend_colors, colorbar=dict(tickvals=numbers, ticktext=names, thickness=25)),
               frames=[go.Frame(data=go.Heatmap(z=tissue_frames_numeric[i])) for i in range(len(tissue_frames_numeric))])
fig.update_layout(
    updatemenus=[
        dict(type="buttons", visible=True,
        buttons=[dict(label="Play", method="animate", args=[None])]
            )])
fig.show()